In [92]:
import pandas as pd
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client.TableView
collection = db["gene2snps"]

In [2]:
collection.count_documents({})

0

In [9]:
df = pd.read_csv("chr6_hic_gene_2_snps_wo_na.txt", sep=" ")

In [10]:
df.head()

,RSID,HomerSig_NHCFV_Regulaotry-Promoter,HomerSig_NHCFV_Promoter-Regulaotry
0,rs1450790813,"RegulatoryBin:chr6:580000:582000,RegulatoryID:...",NaN
1,rs1245863872,"RegulatoryBin:chr6:580000:582000,RegulatoryID:...",NaN
2,rs72835945,"RegulatoryBin:chr6:580000:582000,RegulatoryID:...",NaN
3,rs772278528,"RegulatoryBin:chr6:580000:582000,RegulatoryID:...",NaN
4,rs1038135605,"RegulatoryBin:chr6:580000:582000,RegulatoryID:...",NaN


In [11]:
df.shape

(3026395, 3)

In [89]:
import re
from collections import *
gene2snps = {}

stop = float("inf")
for i, row in df.iterrows():
    rsid = df.iloc[i,0]
    reg = df.iloc[i,1]
    try:
        group = re.findall(r":Gene:([A-Z 0-9 a-z]*)",reg)
        for g in group:
            if g in gene2snps:
                gene2snps[g]["distal_snps"].add(rsid)
            else:
                gene2snps[g] = {}
                gene2snps[g]["distal_snps"] = set([rsid])
                gene2snps[g]["promoter_snps"] = set()
    except:
        pass
    pro = df.iloc[i,2]
    try:
        group = re.findall(r":Gene:([A-Z 0-9 a-z]*)",pro)
        for g in group:
            if g in gene2snps:
                gene2snps[g]["distal_snps"].add(rsid)
            else:
                gene2snps[g] = {}
                gene2snps[g]["distal_snps"] = set()
                gene2snps[g]["promoter_snps"] = set([rsid])
    except:
        pass
    if i > stop:
        break
    if i % 10000 == 0:
        print(f"{round(i / df.shape[0] * 100, 2)} pct is finished")



0.0 pct is finished
0.33 pct is finished
0.66 pct is finished
0.99 pct is finished
1.32 pct is finished
1.65 pct is finished
1.98 pct is finished
2.31 pct is finished
2.64 pct is finished
2.97 pct is finished
3.3 pct is finished
3.63 pct is finished
3.97 pct is finished
4.3 pct is finished
4.63 pct is finished
4.96 pct is finished
5.29 pct is finished
5.62 pct is finished
5.95 pct is finished
6.28 pct is finished
6.61 pct is finished
6.94 pct is finished
7.27 pct is finished
7.6 pct is finished
7.93 pct is finished
8.26 pct is finished
8.59 pct is finished
8.92 pct is finished
9.25 pct is finished
9.58 pct is finished
9.91 pct is finished
10.24 pct is finished
10.57 pct is finished
10.9 pct is finished
11.23 pct is finished
11.56 pct is finished
11.9 pct is finished
12.23 pct is finished
12.56 pct is finished
12.89 pct is finished
13.22 pct is finished
13.55 pct is finished
13.88 pct is finished
14.21 pct is finished
14.54 pct is finished
14.87 pct is finished
15.2 pct is finished
15.5

In [90]:
import pickle
# def serialize_sets(obj):
#     if isinstance(obj, set):
#         return list(obj)
#     return obj
with open("gene2snps.pkl", "wb") as outfile:  
    pickle.dump(gene2snps, outfile) 

In [ ]:
#to load data from the pickle file 

# with open("gene2snps.pkl", "rb") as outfile:  
#     d = pickle.load(outfile) 

In [101]:
def populate(gene2snps):
    for g in gene2snps:
        # this is only to insert SYNGAP1 record
        if g != "SYNGAP1":
            continue
        doc = {
            "gene": g, 
            "promoter_snps": list(gene2snps[g]["promoter_snps"]), 
            "distal_snps": list(gene2snps[g]["distal_snps"]), 
        }
        collection.insert_one(doc)

In [102]:
populate(gene2snps)

In [103]:
collection.count_documents({})

1

In [105]:
len(gene2snps["SYNGAP1"]["distal_snps"])

5262

In [ ]:
len(gene2snps["FOX"]["distal_snps"])